<a href="https://colab.research.google.com/github/hongwon2/BigQuery-Practice/blob/main/GA_Blog_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#날짜별 사용자 수 대비 첫 사용자 비율

In [ ]:
select 
	event_date,
    	count(distinct user_pseudo_id) as user_cnt,
    	count(case when event_name = 'first_visit' then user_pseudo_id else null end) as new_user_cnt,
    	concat(round(100.0 * count(case when event_name = 'first_visit' then user_pseudo_id else null end)/count(distinct user_pseudo_id),0),"%") as new_user_percent
from `analytics_310464334.events_*`
where _TABLE_SUFFIX between '20221009'and'20221013'
group by event_date
order by event_date;

# 최다 조회 사용자의 특성은?

## 최다 조회자는?

In [ ]:
with user_visit_count as (
select 
	user_pseudo_id,
  	count(case when event_name = 'session_start' then 1 else null end) as visit_cnt
from `analytics_310464334.events_*`
where _TABLE_SUFFIX between '20221009'and'20221013'
group by user_pseudo_id
order by visit_cnt desc
)
select *
from user_visit_count

## 본 페이지 추적

In [ ]:
with user_visit_count as (
select user_pseudo_id,
  count(case when event_name = 'session_start' then 1 else null end) as visit_cnt
from `analytics_310464334.events_*`
where _TABLE_SUFFIX between '20221009'and'20221013'
group by user_pseudo_id
order by visit_cnt desc
)
select 
  a.user_pseudo_id,a.visit_cnt,
  b.event_date,
  timestamp_seconds(cast(cast(b.event_timestamp as INT64)/1000000 as INT64)) + interval '9' hour as event_time_stamp,
  b.event_name,
  b.event_params
from user_visit_count as a left join `analytics_310464334.events_*` as b
on a.user_pseudo_id = b.user_pseudo_id 
where a.user_pseudo_id = '2053913881.1650861985' and event_name = 'session_start'

## 본 페이지 추적 (unnest)

In [ ]:
with user_visit_count as (
  select user_pseudo_id,
    count(case when event_name = 'session_start' then 1 else null end) as visit_cnt
  from `analytics_310464334.events_*`
  where _TABLE_SUFFIX between '20221009'and'20221013'
  group by user_pseudo_id
  order by visit_cnt desc
),
unnest_data as (
  select user_pseudo_id,event_date,event_timestamp,event_name,event_params.key,event_params.value
  from `analytics_310464334.events_*`,UNNEST(event_params) AS event_params
  where user_pseudo_id = '2053913881.1650861985' and event_name = 'session_start' and key = 'page_title' 
)
select 
  a.user_pseudo_id,a.visit_cnt,
  b.event_date,
  timestamp_seconds(cast(cast(b.event_timestamp as INT64)/1000000 as INT64)) + interval '9' hour as event_time_stamp,
  b.event_name,
  b.key,
  b.value.string_value
from user_visit_count as a left join unnest_data as b
on a.user_pseudo_id = b.user_pseudo_id
where a.user_pseudo_id = '2053913881.1650861985' and event_name = 'session_start'
order by event_date

## 유저 특성

In [ ]:
select user_pseudo_id,device.category,	device.mobile_brand_name,device.mobile_model_name, device.operating_system,device.operating_system_version,device.language,device.web_info.browser,geo.country
from `analytics_310464334.events_*` 
where user_pseudo_id = '2053913881.1650861985'
limit 1

# 외국에서 접속한 사용자가 본 페이지는?

# 페이지, 페이지 리퍼러 조회

In [ ]:
select 
  user_pseudo_id,
  geo.country,
  geo.region,
  device.language,
  event_name,
  event_params.value.string_value
from `analytics_310464334.events_*`,UNNEST(event_params) AS event_params
where event_name ='page_view' and key in ('page_title','page_referrer') and geo.country not in ('South Korea')

## daum 접속자

In [ ]:
select 
  user_pseudo_id,
  geo.country,
  device.language,
  event_params.key,
  event_params.value.string_value
from `analytics_310464334.events_*`,UNNEST(event_params) AS event_params
where event_name ='page_view' and key ='page_referrer' and value.string_value like '%daum%'
order by event_timestamp